<a href="https://colab.research.google.com/github/Daimond92/python_data_science/blob/main/optimizaci%C3%B3n_aeroportuaria.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**CLASE 1**

https://en.wikipedia.org/wiki/Freedman%E2%80%93Diaconis_rule

#**CLASE 2**

#**CLASE 3**

https://scikit-learn.org/stable/

#**CLASE 4**